<a href="https://colab.research.google.com/github/alfariziraju/Praktek-Fuzzy-G211220061/blob/main/Praktek_fuzzy_Mamdani_G211220061.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Install scikit-fuzzy if not already installed
!pip install scikit-fuzzy
import skfuzzy as fuzz

# --- 1. Domain variabel dan Fungsi Keanggotaan ---

# 1.1 Universe of Discourse
suhu = np.arange(0, 41, 1)        # Suhu (°C)
kelembapan = np.arange(0, 101, 1)  # Kelembapan (%)
kipas = np.arange(0, 101, 1)       # Kecepatan Kipas (%)

# 1.2 Fungsi Keanggotaan Suhu (trimf/segitiga)
suhu_dingin = fuzz.trimf(suhu, [0, 0, 20])
suhu_hangat = fuzz.trimf(suhu, [15, 25, 35])
suhu_panas = fuzz.trimf(suhu, [30, 40, 40])

# 1.3 Fungsi Keanggotaan Kelembapan (trimf/segitiga)
hum_rendah = fuzz.trimf(kelembapan, [0, 0, 40])
hum_sedang = fuzz.trimf(kelembapan, [30, 50, 70])
hum_tinggi = fuzz.trimf(kelembapan, [60, 100, 100])

# 1.4 Fungsi Keanggotaan Kipas (Output) (trimf/segitiga)
kipas_pelan = fuzz.trimf(kipas, [0, 0, 50])
kipas_sedang = fuzz.trimf(kipas, [25, 50, 75])
kipas_cepat = fuzz.trimf(kipas, [50, 100, 100])


# --- 2. Fungsi Fuzzy Mamdani dan Visualisasi ---

def fuzzy_mamdani (suhu_val, kelembapan_val):

    # 2.1 Fuzzifikasi (Menghitung derajat keanggotaan)
    mu_suhu_dingin = fuzz.interp_membership(suhu, suhu_dingin, suhu_val)
    mu_suhu_hangat = fuzz.interp_membership(suhu, suhu_hangat, suhu_val)
    mu_suhu_panas = fuzz.interp_membership(suhu, suhu_panas, suhu_val)

    mu_hum_rendah = fuzz.interp_membership(kelembapan, hum_rendah, kelembapan_val)
    mu_hum_sedang = fuzz.interp_membership(kelembapan, hum_sedang, kelembapan_val)
    mu_hum_tinggi = fuzz.interp_membership(kelembapan, hum_tinggi, kelembapan_val)

    # 2.2 Aturan Fuzzy (Inferensi)
    # AND/OR operator menggunakan np.fmin/np.fmax untuk mendapatkan alpha/bobot aturan.

    # Rule 1: IF Suhu dingin OR Kelembapan rendah -> Kipas pelan
    alpha1 = np.fmax(mu_suhu_dingin, mu_hum_rendah)

    # Rule 2: IF Suhu hangat -> Kipas sedang
    alpha2 = mu_suhu_hangat

    # Rule 3: IF Suhu panas OR Kelembapan tinggi -> Kipas cepat
    alpha3 = np.fmax(mu_suhu_panas, mu_hum_tinggi)

    # 2.3 Implikasi (Clipping)
    # Memotong (clipping) fungsi keanggotaan output dengan nilai alpha/bobot.
    pelan_clipped = np.fmin(alpha1, kipas_pelan)
    sedang_clipped = np.fmin(alpha2, kipas_sedang)
    cepat_clipped = np.fmin(alpha3, kipas_cepat)

    # 2.4 Agregasi Output (Union)
    # Menggabungkan semua fungsi keanggotaan output yang sudah di-clipping.
    aggregated = np.fmax(pelan_clipped, np.fmax(sedang_clipped, cepat_clipped))

    # 2.5 Defuzzifikasi (Menghitung nilai crisp)
    centroid = fuzz.defuzz(kipas, aggregated, 'centroid')
    bisector = fuzz.defuzz(kipas, aggregated, 'bisector')
    mom = fuzz.defuzz(kipas, aggregated, 'mom') # Mean Of Maxima

    # 2.6 Visualisasi
    plt.figure(figsize=(10, 5))

    # Plot fungsi keanggotaan output
    plt.plot(kipas, kipas_pelan, 'b--', label='Pelan')
    plt.plot(kipas, kipas_sedang, 'g--', label='Sedang')
    plt.plot(kipas, kipas_cepat, 'r--', label='Cepat')

    # Plot area yang diagregasi
    plt.fill_between(kipas, aggregated, alpha=0.4, color='orange', label='Aggregated')

    # Plot hasil Defuzzifikasi
    plt.axvline(centroid, color='k', linestyle='-.', label=f'Centroid = {centroid:.1f}')
    plt.axvline(bisector, color='purple', linestyle=':', label=f'Bisector = {bisector:.1f}')
    plt.axvline(mom, color='brown', linestyle='--', label=f'MOM = {mom:.1f}')

    plt.title(f"Suhu={suhu_val:.1f}°C, Kelembapan={kelembapan_val:.1f}%")
    plt.xlabel("Kecepatan Kipas (%)")
    plt.ylabel("Derajat Keanggotaan")
    plt.ylim(-0.05, 1.05)
    plt.grid(True)
    plt.legend(loc='upper left', bbox_to_anchor=(1.01, 1))
    plt.show()

    # 2.7 Print hasil defuzzifikasi
    print("Defuzzifikasi:")
    print(f" Centroid = {centroid:.2f}%")
    print(f" Bisector = {bisector:.2f}%")
    print(f" MOM = {mom:.2f}%")

# --- 3. Jalankan Slider Interaktif ---

# Membuat slider interaktif
interact(
    fuzzy_mamdani,
    # Suhu default 30, range 0-40
    suhu_val=FloatSlider(value=30, min=0, max=40, step=1, description='Suhu (°C)'),
    # Kelembapan default 70, range 0-100
    kelembapan_val=FloatSlider(value=70, min=0, max=100, step=1, description='Kelembapan (%)')
)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 16.5 MB/s eta 0:00:00


interactive(children=(FloatSlider(value=30.0, description='Suhu (°C)', max=40.0, step=1.0), FloatSlider(value=…

<function __main__.fuzzy_mamdani(suhu_val, kelembapan_val)>